In [1]:
using StaticArrays
# using Plots
using LinearAlgebra
# using Makie
using GLMakie 
using LsqFit
include("Elastic.jl")
using .Elastic
using FFMPEG
using DelimitedFiles
using Distributions

In [2]:
# 定义铜的晶格常数（单位：Å）
lattice_constant = 1.0

# 定义铜的FCC晶胞的基矢量
lattice_vectors = collect((Matrix([
    lattice_constant 0.0 0.0; #a1
    0.0 lattice_constant 0.0; #a2
    0.0 0.0 lattice_constant] #a3
))')

# 定义铜的FCC晶胞中的原子位置（单位：Å）
atom_positions = [
    Vector([0.0, 0.0, 0.0]),
    Vector([0.0, 0.5, 0.5]),
    Vector([0.5, 0.0, 0.5]),
    Vector([0.5, 0.5, 0.0]),
    Vector([1.0, 0.0, 0.0]),
    Vector([0.0, 1.0, 0.0]),
    Vector([0.0, 0.0, 1.0]),
    Vector([0.5, 1.0, 0.5]),
    Vector([1.0, 0.5, 0.5]),
    Vector([0.5, 0.5, 1.0]),
    Vector([1.0, 0.0, 1.0]),
    Vector([1.0, 1.0, 0.0]),
    Vector([0.0, 1.0, 1.0]),
    Vector([1.0, 1.0, 1.0])
] 

# 创建铜的原子列表
atoms = [Atom(pos) for pos in atom_positions]



cell=UnitCell(lattice_vectors,atoms)
cpcell=copycell(cell,3,3,3)
fcell=filtercell(cpcell)
fig=visualize_unitcell_atoms(fcell)

In [3]:
#lj势能
function lj(r::Float64)
    return 4*(1/r^12-1/r^6)
end
function Flj(r::Vector{Float64})
    rn=norm(r)
    return 24*(2/rn^14-1/rn^8)*r
end

ct=5.0
interaction = Interaction(lj, Flj, ct, 0.1)

x=1:0.001:ct
y=interaction.cutenergy.(x)
lines(x,y)


In [4]:
visualize_unitcell_atoms0(fcell,iftext=true)
# cell_forcei!(fcell,interaction,1)
println(pressure_int(fcell,interaction))

30.445027174442558


In [5]:
el=Vector{Float64}([])
lattice_constant = 3.61
cl=1.3:0.02:2.5
en=0
fn=0
cp=[3,3,3]
fl=Vector{Float64}([])
pl=Vector{Float64}([])
for lattice_constant in cl
    # 定义铜的FCC晶胞的基矢量
    lattice_vectors = (Matrix([
        lattice_constant 0.0 0.0; #a1
        0.0 lattice_constant 0.0; #a2
        0.0 0.0 lattice_constant] #a3
    ))
    # 创建铜的原子列表
    atoms = [Atom(pos) for pos in atom_positions]

    cell=UnitCell(lattice_vectors,atoms)
    cpcell=copycell(cell,cp...)
    fcell=filtercell(cpcell)
    en=cell_energy0(fcell,interaction,ifnormalize=false)
    push!(el,en)
    fn=sum((force_tensor(fcell,interaction)).^2)*fcell.Volume*27
    push!(fl,fn)
    pn=pressure_int(fcell,interaction)
    push!(pl,pn)
end


In [6]:
minlt=cl[argmin(el)]
println(minlt)
lattice_vectors = (Matrix([
    minlt 0.0 0.0; #a1
    0.0 minlt 0.0; #a2
    0.0 0.0 minlt] #a3
))
atoms = [Atom(pos) for pos in atom_positions]
cell=UnitCell(lattice_vectors,atoms)
cpcell=copycell(cell, cp...)
fcell=filtercell(cpcell)
println(cell_energy0(fcell,interaction,ifnormalize=false))
println(force_tensor(fcell,interaction))
println(pressure_int(fcell,interaction))


1.54
-876.0685054403159
[-0.11788104032228419 -1.4692470074329026e-16 -3.884216226546754e-17; -1.182152764601186e-17 -0.11788104032228489 2.0265475964591762e-17; 4.5034391032426135e-17 3.4901653050130255e-17 -0.11788104032228491]
-0.36307360419263673


In [7]:


function initT!(T::Float64,cell::UnitCell) 
    m=cell.atoms[1].mass
    kb=1.0
    natom=length(cell.atoms)
    sigma=sqrt(kb*T/m)
    Ekstd=3*natom*kb*T/2
    for i in 1:natom
        cell.atoms[i].momentum = rand(Normal(0,sigma),3) 
    end
    Ek=0.0
    for i in 1:natom
        Ek+=0.5*norm(cell.atoms[i].momentum)^2/m
    end

    for i in 1:natom
        cell.atoms[i].momentum=cell.atoms[i].momentum.*sqrt(Ekstd/Ek)
    end
end


function initcell(P::Float64,T::Float64,atoms::Vector{Atom},interaction::Interaction;cp::Vector{Int}=[3,3,3],Prg::Vector{Float64}=[0.05,3.0])
    m=atoms[1].mass
    kb=1.0
    natom=length(atoms)
    a,b=Prg
    for i in 1:100
        lt=a
        lattice_vectors = (Matrix([
                            lt 0.0 0.0; #a1
                            0.0 lt 0.0; #a2
                            0.0 0.0 lt] #a3
                        ))
        cell=UnitCell(lattice_vectors,atoms)
        cpcell=copycell(cell,cp...)
        fcell=filtercell(cpcell)
        initT!(T,fcell)
        pa=pressure_int(fcell,interaction)-P
        
        lt=b
        lattice_vectors = (Matrix([
                            lt 0.0 0.0; #a1
                            0.0 lt 0.0; #a2
                            0.0 0.0 lt] #a3
                        ))
        cell=UnitCell(lattice_vectors,atoms)
        cpcell=copycell(cell,cp...)
        fcell=filtercell(cpcell)
        initT!(T,fcell)
        pb=pressure_int(fcell,interaction)-P
        if pa*pb>0
            throw("Pa,Pb符号相同,Pa=$pa,Pb=$pb")
        end
        c=(a+b)/2
        lt=c
        lattice_vectors = (Matrix([
                            lt 0.0 0.0; #a1
                            0.0 lt 0.0; #a2
                            0.0 0.0 lt] #a3
                        ))
        cell=UnitCell(lattice_vectors,atoms)
        cpcell=copycell(cell,cp...)
        fcell=filtercell(cpcell)
        initT!(T,fcell)
        pc=pressure_int(fcell,interaction)-P
        if abs(pc)<1e-2
            return fcell
        end
        if pa*pc<0
            b=c
        else
            a=c
        end
    end
end

initcell (generic function with 1 method)

In [37]:
# 创建一个新的 Figure
fig = Figure()

# 创建一个 Axis，并设置坐标范围
ax = Axis(fig[1, 1], title = "Line Plot", xlabel = "X-Axis", ylabel = "Y-Axis",
          )
vl=cl.^3*(cpcell.copy[1]*cpcell.copy[2]*cpcell.copy[3]);
# lines!(ax,cl,fl)
lines!(ax,cl,el)
# ylims!(ax, -0.001, 0.001)

fig

In [8]:
# _,fig=BMfit(vl,el,cpcell)
# fig

In [18]:
thermostat = Thermostat(100.0, 1000.0, 0.0, 0.0)
barostat=Barostat(10.0,1000.0,fcell.Volume,0.0)
println(pressure_int(fcell,interaction))
z=cell2z(fcell,thermostat,barostat);

10.003219441492329


In [21]:

# open("z_vectors.txt", "a") do io
#     tpcell=deepcopy(fcell)
#     z=cell2z(tpcell,thermostat,barostat)
#     for i in 1:100                

#         z=RK3_step(z,0.001,tpcell,interaction,thermostat,barostat)
#         tpcell=z2cell(z,tpcell)
#         # println(i)
#         # push!(zl,z)
#         # 将 z 写入文件
#         writedlm(io, z')
#         write(io, "\n")
#         if i%50==0
#             print("$i,")
#         elseif i%2500==0
#             println("$i,")
#         end
#     end  
# end          
zl=[]
tpcell=deepcopy(fcell)
z=cell2z(tpcell,thermostat,barostat)
for i in 1:300              
    RK3_step!(z,0.001,tpcell,interaction,thermostat,barostat)
    push!(zl,deepcopy(z))
    if i%50==0
        print("$i,")
    elseif i%2500==0
        println("$i,")
    end
end  
       

50,100,150,200,250,300,

In [22]:
newcell=z2cell(zl[100],fcell)
println(pressure_int(newcell,interaction))
println(cell_temp(newcell))
# fig=visualize_unitcell_atoms(newcell)
# fig

1.881193130590698
19.225222562074023


In [23]:

pl=[]
Tl=[]
il=1:100
for i in il
    newcell=z2cell(zl[i],fcell)
    p=pressure_int(newcell,interaction)
    T=cell_temp(newcell)
    push!(pl,p)
    push!(Tl,T)
end


In [24]:
fig=Figure(size=(1000,500))
ax1 = Axis(fig[1, 1], title = "P-time", xlabel = "X-Axis", ylabel = "Y-Axis",
          )
ax2=Axis(fig[1, 2], title = "T-time", xlabel = "X-Axis", ylabel = "Y-Axis",
)
lines!(ax1,il,pl)
lines!(ax2,il,Tl)
fig

In [25]:

# 假设func(i)是一个生成3D图的函数
function func(i)
    newcell=z2cell(zl[i],fcell)
    fig=visualize_unitcell_atoms(newcell)
    return fig
end

# function create_animation()
#     video_file = "outputatom.mp4"
#     n_frames = 100
#     fps = 1
#     # 使用record函数生成动画
#     record(fig, video_file, 1:n_frames; framerate = fps) do i
#         fig=func(i)
#         println("Frame $i")
#     end
# end


# # 调用函数生成视频
# create_animation()

func (generic function with 1 method)

In [26]:
outputfold = "outputfold"
isdir(outputfold) || mkpath(outputfold)
for i in 1:100
    fig=func(i)
    save(joinpath(outputfold, "frame$i.png"), fig)
end

In [ ]:
# multi_timestep()

In [9]:
minlt=cl[argmin(el)]
println(minlt)
lattice_vectors = (Matrix([
    minlt 0.0 0.0; #a1
    0.0 minlt 0.0; #a2
    0.0 0.0 minlt] #a3
))
atoms = [Atom(pos) for pos in atom_positions]
for atom in atoms
    atom.momentum = rand(3).*30
 end
cell=UnitCell(lattice_vectors,atoms)
cpcell=copycell(cell, cp...)
fcell=filtercell(cpcell)
println(cell_energy0(fcell,interaction,ifnormalize=false))
println(force_tensor(fcell,interaction))
println(pressure_int(fcell,interaction))
println(cell_temp(fcell))
fcell

1.54
-876.0685054403159
[463.3744720088952 305.40113464336997 476.58883499479595; 305.40113464337 325.7074858649178 380.12954499703784; 476.5888349947959 380.1295449970381 608.550465356204]
232.6346041209715
425.48451521960936


UnitCell([1.54 0.0 0.0; 0.0 1.54 0.0; 0.0 0.0 1.54], Atom[Atom([0.0, 0.0, 0.0], [19.44993325285227, 4.148043598727541, 28.947274456319136], 1.0, 8, [1, 1, 1]), Atom([0.0, 0.0, 1.0], [19.44993325285227, 4.148043598727541, 28.947274456319136], 1.0, 4, [1, 1, 0]), Atom([0.0, 0.0, 2.0], [19.44993325285227, 4.148043598727541, 28.947274456319136], 1.0, 4, [1, 1, 0]), Atom([0.0, 1.0, 0.0], [19.44993325285227, 4.148043598727541, 28.947274456319136], 1.0, 4, [1, 0, 1]), Atom([0.0, 1.0, 1.0], [19.44993325285227, 4.148043598727541, 28.947274456319136], 1.0, 2, [1, 0, 0]), Atom([0.0, 1.0, 2.0], [19.44993325285227, 4.148043598727541, 28.947274456319136], 1.0, 2, [1, 0, 0]), Atom([0.0, 2.0, 0.0], [19.44993325285227, 4.148043598727541, 28.947274456319136], 1.0, 4, [1, 0, 1]), Atom([0.0, 2.0, 1.0], [19.44993325285227, 4.148043598727541, 28.947274456319136], 1.0, 2, [1, 0, 0]), Atom([0.0, 2.0, 2.0], [19.44993325285227, 4.148043598727541, 28.947274456319136], 1.0, 2, [1, 0, 0]), Atom([1.0, 0.0, 0.0], [1

In [10]:
function update_celli!(rl,pl,i::Int,cell::UnitCell)
    cell.atoms[i].position=inv(cell.lattice_vectors)*rl
    cell.atoms[i].momentum=pl
end

function LA_step!(fcell::UnitCell,interaction::Interaction,barostat::Barostat,n::Int,dt::Float64,T::Float64;gamma0::Float64=1.0,gammav::Float64=1.0)
kb=1.0
dist=Normal(0,1)

Pe=barostat.Pe
ddt=dt/n
natom=length(fcell.atoms)
pl=fill(zeros(3),(2*n+1,natom))
rl=fill(zeros(3),(2*n+1,natom))
Pint=pressure_int(fcell,interaction)
Rv=barostat.Pv


Rvdt2=Rv+(Pint-Pe)*dt/2+sqrt(kb*T*gammav*dt)*rand(dist)-gammav*Rv*dt/2/barostat.W



V0=fcell.Volume
Vdt2=V0+Rvdt2*dt/2/barostat.W
Vdt=Vdt2+Rvdt2*dt/2/barostat.W
# println("$Rv,$(Rvdt2*dt/2/barostat.W)")
mi=fcell.atoms[1].mass
fdt2=fill(zeros(3),natom)
fcell.Volume=Vdt
barostat.V=Vdt
fcell.lattice_vectors=fcell.lattice_vectors.*(Vdt/V0)^(1/3)
ltv=fcell.lattice_vectors


for i in 1:natom
    pl[1,i]=fcell.atoms[i].momentum
    rl[1,i]=ltv*fcell.atoms[i].position
end
for j in 1:n-1
    fl=fill(zeros(3),length(fcell.atoms))
    for i in 1:length(fcell.atoms)
        fi=cell_forcei(fcell,interaction,i)
        fl[i]=fi
    end
    for i in 1:length(fcell.atoms)
        if j==1
            pl[2*n+1,i]=pl[1,i].+(dt/2)*fl[i].+sqrt(kb*T*gamma0*dt)*rand(dist,3)-(gamma0*dt/2/mi)*pl[1,i]
        end
        pl[2*j,i]=pl[j,i]+(ddt/2).*fl[i]+sqrt(kb*T*gamma0*ddt)*rand(dist,3)-(gamma0*ddt/2/mi)*pl[j,i]
        rl[2*j+1,i]=rl[2*j-1,i]+(ddt/2).*pl[2*j,i]*ddt/mi
        update_celli!(rl[2*j+1,i],pl[2*j,i],i,fcell)
    end

    fl=fill(zeros(3),length(fcell.atoms))
    for i in 1:length(fcell.atoms)
        fi=cell_forcei(fcell,interaction,i)
        fl[i]=fi
    end

    for i in 1:length(fcell.atoms)
        pl[2*j+1,i]=pl[2*j,i]+ddt/2*fl[i]+sqrt(kb*T*gamma0*ddt)*rand(dist,3)-(gamma0*ddt/2/mi)*pl[2*j,i]
    end   
end
for i in 1:length(fcell.atoms)
    mi=fcell.atoms[i].mass
    rl[2*n+1,i]=rl[2*n-1,i]+((V0/Vdt2)^(2/3)/mi*ddt).*pl[2*n,i]
    rl[2*n+1,i]=rl[2*n+1,i].*((Vdt/V0)^(1/3))
    pl[n+1,i]=pl[n+1,i].*((V0/Vdt)^(1/3))
end

invlt=inv(fcell.lattice_vectors)
for i in 1:length(fcell.atoms)
    fcell.atoms[i].position=invlt*rl[2*n+1,i]
end

for i in 1:length(fcell.atoms)
    fi=cell_forcei(fcell,interaction,i)
    pl[2*n+1,i]=pl[n+1,i]+(dt/2).*fi-(gamma0*dt/2/mi)*pl[n+1,i]+sqrt(kb*T*gammav*dt)*rand(dist,3)
end

for i in 1:length(fcell.atoms)
    fcell.atoms[i].momentum=pl[2*n+1,i]
end

Pint=pressure_int(fcell,interaction)
Rvdt=Rvdt2+(Pint-Pe).*(dt/2)+sqrt(kb*T*gammav*dt)*rand(dist)-gammav*Rvdt2*dt/2/barostat.W
barostat.Pv=Rvdt
end

LA_step! (generic function with 1 method)

0.9958098807883018
0.9999999999999998


UnitCell([0.6783849048614501 0.0 0.0; 0.0 0.6783849048614501 0.0; 0.0 0.0 0.6783849048614501], Atom[Atom([0.0, 0.0, 0.0], [-1.2730889451585652, -1.0192789914891538, 1.0642174243263576], 1.0, 8, [1, 1, 1]), Atom([0.0, 0.0, 1.0], [-0.677319940280466, 0.8322679567973591, 0.9145214956122028], 1.0, 4, [1, 1, 0]), Atom([0.0, 0.0, 2.0], [-0.1516209408826949, -1.2341927266432957, 0.5434335240853937], 1.0, 4, [1, 1, 0]), Atom([0.0, 1.0, 0.0], [-0.3503901239072071, -1.1354666909987114, -0.17398109701588785], 1.0, 4, [1, 0, 1]), Atom([0.0, 1.0, 1.0], [-0.8112116147910783, -1.7008047138532445, -2.2596213247352295], 1.0, 2, [1, 0, 0]), Atom([0.0, 1.0, 2.0], [0.6194580935070234, 0.4412575300980543, 1.2292644956995573], 1.0, 2, [1, 0, 0]), Atom([0.0, 2.0, 0.0], [-1.1705140103413527, 0.0055960506512460215, -0.9485387071284419], 1.0, 4, [1, 0, 1]), Atom([0.0, 2.0, 1.0], [0.7166023082858238, -1.3641548999420632, 0.7468063258660749], 1.0, 2, [1, 0, 0]), Atom([0.0, 2.0, 2.0], [0.2196578096177579, 0.114755

In [16]:
fcell=initcell(10.0,10.0,atoms,interaction,cp=[3,3,3],Prg=[0.03,8])
println(pressure_int(fcell,interaction))
println(cell_temp(fcell))
fcell
totalmass=0.0
for atom in fcell.atoms
    totalmass+=atom.mass
end
println(totalmass)
println(fcell.Volume)

9.99289841788624
9.999999999999996
108.0
8.793708485639089


In [ ]:
barostat=Barostat(10.0,100000,fcell.Volume,0.0)
for i in 1:1000

    LA_step!(fcell,interaction,barostat,4,0.0001,1.0,gamma0=10.0,gammav=10.0)
    if i%1==0
        println(pressure_int(fcell,interaction))
        # println(fcell.Volume)
        # println(fcell.lattice_vectors)
        println(barostat.V)
        println(barostat.Pv)
        println(cell_temp(fcell))
        println()
    end
end

50.61015707359302
8.793708527050308
0.08306982649461986
8.32615098749947

50.54626478373265
8.79370852712586
0.10894618084670879
8.320090768951616

50.41681474640583
8.793708527218682
0.09009977033013483
8.30345672767243

50.168525397083215
8.793708527295104
0.08303971350029314
8.267623716940431

50.308059801363264
8.793708527444553
0.17261138470246237
8.27161646611517

50.060275312388114
8.793708527601522
0.1006687490838478
8.269181453006885

49.94079586102006
8.793708527680787
0.06096908392988981
8.254172686412765

49.58122762295886
8.793708527759078
0.11768324744946858
8.233003267048925

49.875295667194614
8.793708527815182
0.023354000094138544
8.22799060692925

49.77313242029945
8.793708527876026
0.0261352413273793
8.215144357795946

49.80623533316236
8.793708527839557
-0.058529182898667695
8.218268512956191

49.71968280745377
8.793708527793209
-0.05347218607445913
8.201657720518373

49.794528866270646
8.793708527762755
-0.053345508425702355
8.183902442229208

49.85864203116958
8.7

[32680:24932:1009/151058.026:ERROR:t_skinfilemap.cpp(63)] SGRenderLog:CheckUpdate newMem.OpenMem Suc, szNewMem:Local\sgime_skin.Dpi144.v3.filemap.sogouime
[32680:24932:1009/151058.026:ERROR:t_skinfilemap.cpp(65)] SGRenderLog:CheckUpdate finish: old version:1
[32680:24932:1009/151058.026:ERROR:t_skinfilemap.cpp(67)] SGRenderLog:CheckUpdate finish: new version:3
[32680:24932:1009/151058.099:ERROR:t_skinfilemap.cpp(63)] SGRenderLog:CheckUpdate newMem.OpenMem Suc, szNewMem:Local\sgime_skin.Dpi144.v3.filemap.sogouime
[32680:24932:1009/151058.099:ERROR:t_skinfilemap.cpp(65)] SGRenderLog:CheckUpdate finish: old version:3
[32680:24932:1009/151058.099:ERROR:t_skinfilemap.cpp(67)] SGRenderLog:CheckUpdate finish: new version:3


41.81980736844811
8.793708540693961
0.5686898725049119
7.12153839578483

41.770276080965345
8.793708541246746
0.5284700880768088
7.116812147632548

41.438607514143186
8.793708541779326
0.5233233383189305
7.089577254015938

41.38872326457728
8.793708542272183
0.4791934364811179
7.084623110592734

41.24154896602105
8.793708542704033
0.444282693800234
7.063378910269355

41.00408770934181
8.793708543135045
0.44920156884919565
7.021401395663752

40.606005096505285
8.793708543578566
0.4277839868875891
6.99276144965835

40.46245570205809
8.793708544004092
0.49925102248522807
6.972385718181971

40.391527699295544
8.793708544508009
0.45421161332629856
6.963919714033412

40.12718795218765
8.79370854498634
0.5010811157822366
6.923789425660364

39.963009446504266
8.79370854549139
0.522822902277884
6.899947327473434

39.87292766184004
8.793708545992144
0.5241834753098547
6.882094050167899

39.7951414077033
8.793708546507292
0.565279509439347
6.872476801894903

39.664063132622275
8.79370854705205
0.

In [15]:
pressure_int1(fcell,interaction)

(122.6942363391321, 1.025517714912756)

In [14]:
function pressure_int1(cell::UnitCell,interaction::Interaction,dUdV::T=dUdV_default) where T
    v=cell.Volume
    Pint1=0.0
    Pint2=0.0
    lt=cell.lattice_vectors
    for i in 1:length(cell.atoms)
        atom=cell.atoms[i]
        pm=atom.momentum
        ri=lt*atom.position
        fi=cell_forcei(cell,interaction,i)
        Pint1+=dot(pm,pm)/atom.mass
        Pint2+=dot(ri,fi)
    end
    
    return Pint1/v/3,Pint2/v/3
end

pressure_int1 (generic function with 2 methods)